In [8]:
# add for tensorboard code
from __future__ import absolute_import, division, print_function, unicode_literals
import argparse
import json
from tensorflow.python.lib.io import file_io

# origin fairing code
import tensorflow as tf
import os

class MyFashionMnist(object):
  def train(self):
           
    print("TensorFlow version: ", tf.__version__)

    mnist = tf.keras.datasets.mnist

    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.summary()

    print("Training...")
    
    model.fit(x_train, y_train, epochs=3, validation_split=0.2 ) 
    
    score = model.evaluate(x_test, y_test, batch_size=128, verbose=0)
    print('Test accuracy: ', score[1])

if __name__ == '__main__':
    if os.getenv('FAIRING_RUNTIME', None) is None:
        from kubeflow import fairing
        from kubeflow.fairing.kubernetes import utils as k8s_utils
        
        DOCKER_REGISTRY = 'kubeflow-registry.default.svc.cluster.local:30000'   # private registry

        fairing.config.set_builder(
            'append',
            image_name='tensorboard-job', # here not fairing job but katib job
            base_image='brightfly/kubeflow-jupyter-lab:tf2.0-cpu',
            registry=DOCKER_REGISTRY, 
            push=True
        )
        # cpu 1, memory 5GiB
        fairing.config.set_deployer('job',
                                    namespace='admin', # here
                                    pod_spec_mutators=[
                                        k8s_utils.get_resource_mutator(cpu=1,  # here
                                                                       memory=5)]
                                   )
        fairing.config.run()
    else:
        remote_train = MyFashionMnist()
        remote_train.train()

[I 201024 12:40:10 config:123] Using preprocessor: <kubeflow.fairing.preprocessors.converted_notebook.ConvertNotebookPreprocessor object at 0x7f970cd00b70>
[I 201024 12:40:10 config:125] Using builder: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7f970cbc4f98>
[I 201024 12:40:10 config:127] Using deployer: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7f970cbc4f98>
[W 201024 12:40:10 append:50] Building image using Append builder...
[I 201024 12:40:10 base:105] Creating docker context: /tmp/fairing_context_t0w9s4t1
[I 201024 12:40:10 converted_notebook:127] Converting train-fairing.ipynb to train-fairing.py
[I 201024 12:40:10 docker_creds_:234] Loading Docker credentials for repository 'brightfly/kubeflow-jupyter-lab:tf2.0-cpu'
[W 201024 12:40:12 append:54] Image successfully built in 2.2520419789943844s.
[W 201024 12:40:12 append:94] Pushing image kubeflow-registry.default.svc.cluster.local:30000/tensorboard-job:2FA46D7F...
[I 201024 12:40:12

TensorFlow version:  2.0.0
11493376/11490434 [==============================] - 0s 0us/step
2020-10-24 12:40:18.065156: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2020-10-24 12:40:18.071885: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000140000 Hz
2020-10-24 12:40:18.072719: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x553ddc0 executing computations on platform Host. Devices:
2020-10-24 12:40:18.072916: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Host, Default Version
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #
flatten (Flatten)            (None, 784)               0
_________________________________________________________________
dense (Dense)                (None, 128)               10048

[W 201024 12:41:01 job:162] Cleaning up job fairing-job-bc5xp...
